# Portfolio Information - Demo
This notebook demonstrates how to extract data and calculates the Portfolio Information summary using dfirb demo data. 

Please refer to the documentation of the *crv_pylib* for details on the applied functions: */crv_pylib/docs/html/index.html*

#### Load libraries & set options

In [1]:
# imports and options
import pandas as pd
from crv.validation.summary import pd_portfolio_information

from IPython.display import display
pd.options.display.max_columns = None

Invoking __init__.py for crv.utils


## Step 1. Load & transform demo data.

Change path to data as follows: "C:\\Users\\<<*your_user_id*>>\\Nordea\\CRV_PYLIB - Documents\\Testing Notebooks\\test_data\\"

In [2]:
df_sco = pd.DataFrame()
infile = "dfirb_2016_2017.pkl"
print(infile)
#path = "\\\\VDA1cs4756\\DEV_MODEL\\CRMV\\abt\\"
user_id = "G85544"
path = "C:\\Users\\" + user_id + "\\Nordea\\CRV_PYLIB - Documents\\Testing Notebooks\\test_data\\"
dfirb = pd.read_pickle(path + infile)
print(type(dfirb))
print(dfirb.shape) ## (1093864, 33)

## Add a default column & separate 2016 - 2017
dfirb['df_2'] = dfirb.FRATINGO.apply(lambda rating: 1 if '0' in rating else 0)
dfobs = dfirb[dfirb.CALMONTH=='201612']
dfperf = dfirb[dfirb.CALMONTH=='201712']
print(dfobs.shape) ## (473105, 34)
print(dfperf.shape) ## (620759, 34)

## Group & aggregate by customer ID : group
d = {'BA_1RWAAR':'sum',
     'EAD':'sum',
     'df_2':'first',
     'rating':'first'}
df1 = dfobs[['group', 'rating', 'df_2','BA_1RWAAR','EAD']].copy()
df1 = df1.sort_values(by=['group', 'rating']).groupby('group',observed = True,as_index=False).agg(d)
print(df1.shape) ## (32286, 5)
###df1 = df1[~df1.rating.str.contains('0')]  # uncomment to exlcude zero ratings
df2 = dfperf[['group', 'rating', 'df_2','BA_1RWAAR','EAD']].copy()
df2 = df2.sort_values(by=['group', 'rating']).groupby('group',observed = True,as_index=False).agg(d)
print(df2.shape) ## (116857, 5)
###df2 = df2[~df2.rating.str.contains('0')]  # uncomment to exclude zero ratings

dfirb_2016_2017.pkl
<class 'pandas.core.frame.DataFrame'>
(1093864, 33)
(473105, 34)
(620759, 34)
(32286, 5)
(116857, 5)


## Step 2. Calculate portfolio information

In [3]:
# calculate the porfolio information
pdinf = pd_portfolio_information(df1=df1,
                                df2=df2,
                                d_colnames={'RWEA': 'BA_1RWAAR',
                                            'EAD': 'EAD', 
                                            'cust_id': 'group',
                                            'rating': 'rating',
                                            'default': 'df_2'})
display(pdinf)

Measure  \
0                                    RWEA   
1                                     EAD   
2                     Number of customers   
3                 Number of rating grades   
4  Exposure value of customers in default   
5                      Number of defaults   

   Beginning of the observation period  End of the observation period  \
0                         6.901585e+10                   6.593740e+10   
1                         2.020449e+11                   1.829565e+11   
2                         3.228600e+04                   1.168570e+05   
3                         2.100000e+01                   2.100000e+01   
4                         4.491561e+09                   7.229808e+09   
5                         1.392000e+03                   8.764100e+04   

         Change  
0 -3.078452e+09  
1 -1.908847e+10  
2  8.457100e+04  
3  0.000000e+00  
4  2.738248e+09  
5  8.624900e+04